In [1]:
from videolabeler import utils as vl
import numpy as np
import cv2
from tqdm.notebook import tqdm
import pandas as pd
import os
import random


In [19]:
input_file = '/home/mmiller/Documents/data/imuNet_data/master_labels.csv'

input_df = pd.read_csv(input_file, index_col=0)
input_df.head()



/home/mmiller/Applications/anaconda3/envs/video-labeler-linux/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DC,KZ,MM,MT,OM,SG,SN,animal_id,frame,time
0,NaN,NaN,explore,NaN,NaN,NaN,NaN,V3-154,19800.0,NaN
1,NaN,NaN,explore,NaN,NaN,NaN,NaN,V3-154,19801.0,NaN
2,NaN,NaN,explore,NaN,NaN,NaN,NaN,V3-154,19802.0,NaN
3,NaN,NaN,right turn [still],NaN,NaN,NaN,NaN,V3-154,19803.0,NaN
4,NaN,NaN,right turn [still],NaN,NaN,NaN,NaN,V3-154,19804.0,NaN


In [20]:
animals = input_df.animal_id.unique()
print(animals,'\n')

# remove erroneous IDs from animals:
animals = [i for i in animals if isinstance(i, str)]
animals = [i for i in animals if len(i) > 1]
animals = [i for i in animals if len(i) < 25]

print(animals)

#how many frames are there in each recording
#rec_dict = {i:len(input_df.loc[input_df.animal_id == i].frame.unique()) for i in animals}
rec_dict = {i : np.arange(int(np.nanmin(input_df.loc[input_df.animal_id == i].frame.values)), int(np.nanmax(input_df.loc[input_df.animal_id == i].frame.values))) for i in animals}

output_df = pd.DataFrame()

print(rec_dict)

['V3-154' 'V' 'V3-163' nan '2020-03-20 13:17:32.677047' 'DCMStr41'
 'uG58-131' 'V3-166' 'OM047' 'OM035' 'OM035_0402_exp' 'OM051_0402_exp'
 'OM038_0415_pref' 'OM047_0415_pref'] 

['V3-154', 'V3-163', 'DCMStr41', 'uG58-131', 'V3-166', 'OM047', 'OM035', 'OM035_0402_exp', 'OM051_0402_exp', 'OM038_0415_pref', 'OM047_0415_pref']
{'V3-154': array([    0,     1,     2, ..., 22816, 22817, 22818]), 'V3-163': array([    0,     1,     2, ..., 23421, 23422, 23423]), 'DCMStr41': array([    0,     1,     2, ..., 23532, 23533, 23534]), 'uG58-131': array([    0,     1,     2, ..., 30928, 30929, 30930]), 'V3-166': array([    0,     1,     2, ..., 34341, 34342, 34343]), 'OM047': array([   0,    1,    2, ..., 5997, 5998, 5999]), 'OM035': array([   0,    1,    2, ..., 5996, 5997, 5998]), 'OM035_0402_exp': array([1301, 1302, 1303, ..., 5996, 5997, 5998]), 'OM051_0402_exp': array([ 401,  402,  403, ..., 5446, 5447, 5448]), 'OM038_0415_pref': array([ 6400,  6401,  6402, ..., 17996, 17997, 17998]), 'OM047_0415

## Preprocessing:

In [21]:
#this custom aggregate function finds the first non-NA label in a column and returns it for the
#merged row. If the same labeler labeled twice (eg after being served two consecutive batches), it takes their first label
def label_exists(col):
    for row in col:
        if pd.isna(row) == False:
            return row

    return None

#consolidate all overlapping frames into one row
input_df = input_df.groupby(['frame', 'animal_id']).aggregate(label_exists).reset_index()
input_df.sort_values(by='frame')

,frame,animal_id,DC,KZ,MM,MT,OM,SG,SN,time
0,0.0,DCMStr41,None,None,None,walking,None,None,None,2020-03-23 12:14:19.007276
1,0.0,OM035,None,None,None,None,0.0,None,None,2020-04-30 18:14:05.089107
2,0.0,OM047,None,walking,None,None,None,None,None,2020-04-30 10:37:13.532376
3,0.0,V3-154,None,None,None,explore,None,None,None,2020-03-20 15:58:59.088427
4,0.0,V3-163,walking,None,None,None,None,None,None,2020-03-26 08:46:41.259802
...,...,...,...,...,...,...,...,...,...,...
176252,34340.0,V3-166,explore,None,None,None,None,None,None,2020-04-01 10:30:09.861308
176253,34341.0,V3-166,left turn [still],None,None,None,None,None,None,2020-04-01 10:30:09.861308
176254,34342.0,V3-166,left turn [still],None,None,None,None,None,None,2020-04-01 10:30:09.861308
176255,34343.0,V3-166,left turn [still],None,None,None,None,None,None,2020-04-01 10:30:09.861308


In [ ]:
# What is this code block doing?

#output = '/ariel/science/mmiller/data/imu_net/tiff_to_label/master_labels_V3154_merged.csv'

#input_df.to_csv(output)




## Consensus generation:

In [22]:
labelers = [i for i in input_df.columns if len(i) == 2]

In [25]:
subj_ids = ['OM047_0415_pref']

In [26]:
output_df = pd.DataFrame()
#for each recording

for animal in subj_ids:
    print(animal)
    con_labels = []
    labeler_ids = []
    
    #for each labeler, make a list of which frames have nan values so we can find the closest nan easily later
    nan_dict = {col:input_df.loc[(input_df.animal_id == animal) & (input_df[col].isna())].frame.values for col in labelers}
    
    num_frames = rec_dict[animal]
    
    #for frame in range(num_frames):
    for frame in num_frames:
        labels = input_df.loc[(input_df.animal_id == animal) & (input_df.frame == frame)]
        
        #for each frame, drop labelers who didn't label it
        labels = labels[labelers].dropna(axis=1)
        #print(labels['MT'].iloc[0])
        
        
        #if no overlap, simply use the existing label
        if labels.shape[1] == 1:
            con_labels.append(labels[labels.columns[0]].iloc[0])
            labeler_ids.append(labels.columns[0])
            
        #if there is overlap    
        elif labels.shape[1] > 1:
            labeler = None
            
            #The normal overlap case: for each labeler, check if they're the 1st labeler of 1st half or 2nd labeler of 2nd
            for col in labels.columns:
                
                closest_nan_gap = abs(min([nan_frame - frame for nan_frame in nan_dict[col]], key=abs)) 
                
                #gap of more than n_overlap/2 between current frame and closest nan means
                #this labeler is either the first labeler of first half or second labeler of second half
                if (closest_nan_gap >= 25) & (closest_nan_gap <= 50):
                    labeler = col
                    break
                
                
            #if you didn't meet the criteria set after going through all the columns, 
            #(i.e due to total overlaps and edge cases etc), just pick the label of whoever labeled the frame before
            #this is so we don't have chunks of low-consensus windows due to randomly interleaving labelers
            if labeler == None and labeler_ids[-1] in labels.columns and len(labeler_ids) != 0:
                labeler = labeler_ids[-1]
                
                
            #if you fail even that (ie because it's the start of a batch), just go random. 
            #Only V3-154 should ever have to reach this    
            elif labeler == None:
                labeler = random.choice(labels.columns)
                
            
            #finally, add your labels
            con_labels.append(labels[labeler].iloc[0])
            labeler_ids.append(labeler)
                
            
    #animal_df = pd.DataFrame({'animal_id': [animal]*num_frames, 'frame': np.arange(num_frames), 
    #                          'label':con_labels, 'labeler': labeler_ids})
    animal_df = pd.DataFrame({'animal_id': [animal]*len(num_frames), 'frame': num_frames, 
                              'label':con_labels, 'labeler': labeler_ids})        
    
    
    output_df = output_df.append(animal_df)

OM047_0415_pref


In [27]:
print(len(con_labels))
print(len(num_frames))
print(len(labeler_ids))

11599
11599
11599


In [28]:
animal_df.head()

,animal_id,frame,label,labeler
0,OM047_0415_pref,6400,0.0,DC
1,OM047_0415_pref,6401,0.0,DC
2,OM047_0415_pref,6402,0.0,DC
3,OM047_0415_pref,6403,0.0,DC
4,OM047_0415_pref,6404,0.0,DC


In [29]:
out_fn = '/home/mmiller/Documents/data/astellas/free_social/labels_OM047_0415_pref_consensus.csv'

In [30]:
animal_df.to_csv(out_fn)

In [ ]:
output_df.to_csv('/ariel/science/mmiller/data/imu_net/tiff_to_label/all-consensus-test.csv')

## Spot check

In [ ]:
video_dir = '/ariel/science/mmiller/data/imu_net/tiff_to_label/V3-154/'
labels_file = '/ariel/science/mmiller/data/imu_net/tiff_to_label/V3-154-consensus-new-method.csv'

for i in range(10):
    vl.window_and_inspect_tiff(video_dir, labels_file)